In [1]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, precision_recall_curve
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import seaborn as sns

# %%
# =============================================================================
# PART 1: DATA LOADING & CLEANING
# =============================================================================
print("="*80)
print("PART 1: DATA LOADING & CLEANING")
print("="*80)

df = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=['Tanggal'])
df = df.sort_values('Tanggal').reset_index(drop=True)

print(f"Initial data shape: {df.shape}")

# Check and fix strip values
checkstripvalue = (df['Curah Hujan (mm)'] == "-").sum()
print(f"Strip values found: {checkstripvalue}")

df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].replace('-', 0)
df['Curah Hujan (mm)'] = df['Curah Hujan (mm)'].astype(float)

checkstripvalue2 = (df['Curah Hujan (mm)'] == 8888).sum()
print(f"8888 values found: {checkstripvalue2}")

# %%
# =============================================================================
# PART 2: ADVANCED MISSING VALUE IMPUTATION
# =============================================================================
print("\n" + "="*80)
print("PART 2: MISSING VALUE IMPUTATION WITH RANDOM FOREST")
print("="*80)

df2 = pd.read_excel("Updated_Data_Historis_2015_2025.xlsx", parse_dates=["Tanggal"])
df2 = df2.sort_values("Tanggal").reset_index(drop=True)

# Replace all invalid values
invalid_values = ["-", " - ", "–", "—", "N/A", "n/a", "", " ", "8888", 8888]
df2 = df2.replace(invalid_values, np.nan)
df2 = df2.replace(r'^\s*$', np.nan, regex=True)

# Handle wind direction
col = "Arah Angin Terbanyak (°)"
df2[col] = (df2[col].astype(str).str.strip().str.upper()
            .str.replace("–", "-", regex=False)
            .str.replace("—", "-", regex=False))

mapping = {
    "C": 0, "N": 0, "NE": 45, "E": 90, "SE": 135,
    "S": 180, "SW": 225, "W": 270, "NW": 315
}
df2[col] = df2[col].map(mapping)

# Convert to numeric
for c in df2.columns:
    if c != "Tanggal":
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

target = "Curah Hujan (mm)"

# Check correlation before imputation
numeric_df = df2.select_dtypes(include=[np.number])
target_corr = numeric_df.corr()[target].sort_values(ascending=False)
print("\nTop 10 correlations with target:")
print(target_corr.head(10))

# Impute using Random Forest
feature_cols = [c for c in df2.columns if c not in ["Tanggal", target]]
df2[feature_cols] = df2[feature_cols].fillna(df2[feature_cols].median())

train_df = df2[df2[target].notna()]
pred_df = df2[df2[target].isna()]

X_train = train_df[feature_cols]
y_train = train_df[target]
X_pred = pred_df[feature_cols]

print(f"\nImputing {len(pred_df)} missing target values...")

from sklearn.ensemble import RandomForestRegressor
rf_imputer = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
rf_imputer.fit(X_train, y_train)

df2.loc[df2[target].isna(), target] = rf_imputer.predict(X_pred)

df2.to_excel("Ready_To_Use_Updated_RF.xlsx", index=False)
print("✓ File saved: Ready_To_Use_Updated_RF.xlsx")
print(f"✓ Missing values after imputation: {df2.isna().sum().sum()}")

# %%
# =============================================================================
# PART 3: ADVANCED FEATURE ENGINEERING
# =============================================================================
print("\n" + "="*80)
print("PART 3: ADVANCED FEATURE ENGINEERING")
print("="*80)

df3 = pd.read_excel("Ready_To_Use_Updated_RF.xlsx")
df3['Tanggal'] = pd.to_datetime(df3['Tanggal'])
df3 = df3.sort_values('Tanggal').reset_index(drop=True)

# Save original target
df3['Curah Hujan Old'] = df3['Curah Hujan (mm)'].copy()

# 1. LAG FEATURES
print("Creating lag features...")
lags = [1, 3, 7]
lag_columns = [
    "Curah Hujan (mm)",
    "Kelembapan Rata-rata",
    "Temperatur Rata-rata",
    "Temperatur Maksimum",
    "Temperatur Minimum",
    "Lamanya Penyinaran Matahari",
    "Kecepatan Angin Rata-rata"
]

for col in lag_columns:
    for lag in lags:
        df3[f"{col}_lag_{lag}"] = df3[col].shift(lag)

# 2. ROLLING STATISTICS
print("Creating rolling statistics...")
# Rolling mean
df3["Rain_roll_3"] = df3["Curah Hujan (mm)"].shift(1).rolling(3).mean()
df3["Rain_roll_7"] = df3["Curah Hujan (mm)"].shift(1).rolling(7).mean()
df3["Humidity_roll_3"] = df3["Kelembapan Rata-rata"].shift(1).rolling(3).mean()
df3["Humidity_roll_7"] = df3["Kelembapan Rata-rata"].shift(1).rolling(7).mean()
df3["Temp_Avg_roll_3"] = df3["Temperatur Rata-rata"].shift(1).rolling(3).mean()
df3["Temp_Avg_roll_7"] = df3["Temperatur Rata-rata"].shift(1).rolling(7).mean()
df3["Temp_Max_roll_3"] = df3["Temperatur Maksimum"].shift(1).rolling(3).mean()
df3["Temp_Max_roll_7"] = df3["Temperatur Maksimum"].shift(1).rolling(7).mean()
df3["Temp_Min_roll_3"] = df3["Temperatur Minimum"].shift(1).rolling(3).mean()
df3["Temp_Min_roll_7"] = df3["Temperatur Minimum"].shift(1).rolling(7).mean()
df3["Sunshine_roll_3"] = df3["Lamanya Penyinaran Matahari"].shift(1).rolling(3).mean()
df3["Sunshine_roll_7"] = df3["Lamanya Penyinaran Matahari"].shift(1).rolling(7).mean()
df3["Wind_roll_3"] = df3["Kecepatan Angin Rata-rata"].shift(1).rolling(3).mean()
df3["Wind_roll_7"] = df3["Kecepatan Angin Rata-rata"].shift(1).rolling(7).mean()

# EWM (Exponentially Weighted Moving Average)
df3['Rain_ewm_3'] = df3['Curah Hujan (mm)'].shift(1).ewm(span=3).mean()
df3['Rain_ewm_7'] = df3['Curah Hujan (mm)'].shift(1).ewm(span=7).mean()

# Rolling std
df3['Rain_roll_std_7'] = df3['Curah Hujan (mm)'].shift(1).rolling(7).std()
df3['Temp_roll_std_7'] = df3['Temperatur Rata-rata'].shift(1).rolling(7).std()

# 3. SEASONAL FEATURES
print("Creating seasonal features...")
df3['month'] = df3['Tanggal'].dt.month
df3['day_of_year'] = df3['Tanggal'].dt.dayofyear
df3['week_of_year'] = df3['Tanggal'].dt.isocalendar().week

# Season mapping (Indonesia: Dry season Apr-Oct, Wet season Nov-Mar)
df3['season'] = df3['month'].map({
    12:1, 1:1, 2:1, 3:1,  # Wet season
    4:2, 5:2, 6:2, 7:2, 8:2, 9:2, 10:2, 11:2  # Dry season
})

# Cyclical encoding for month
df3['month_sin'] = np.sin(2 * np.pi * df3['month'] / 12)
df3['month_cos'] = np.cos(2 * np.pi * df3['month'] / 12)

# 4. INTERACTION FEATURES
print("Creating interaction features...")
df3['temp_humidity'] = df3['Temperatur Rata-rata'] * df3['Kelembapan Rata-rata']
df3['temp_range'] = df3['Temperatur Maksimum'] - df3['Temperatur Minimum']
df3['humidity_temp_ratio'] = df3['Kelembapan Rata-rata'] / (df3['Temperatur Rata-rata'] + 1)

# 5. DRY SPELL COUNTER
print("Creating dry spell counter...")
df3['is_dry'] = (df3['Curah Hujan (mm)'] == 0).astype(int)
df3['dry_spell'] = df3['is_dry'].groupby((df3['is_dry'] != df3['is_dry'].shift()).cumsum()).cumsum()

# 6. RAIN INTENSITY CATEGORIES
print("Creating rain intensity features...")
df3['rain_yesterday'] = (df3['Curah Hujan (mm)'].shift(1) > 0).astype(int)
df3['rain_last_3days'] = (df3['Curah Hujan (mm)'].shift(1).rolling(3).sum() > 0).astype(int)

# Drop rows with NaN (due to lag/rolling operations)
df3_fe = df3.dropna().reset_index(drop=True)

print(f"\n✓ Feature engineering complete!")
print(f"✓ Final shape: {df3_fe.shape}")
print(f"✓ Number of features: {df3_fe.shape[1] - 2}")  # Exclude Tanggal and target

df3_fe.to_excel("df3_feature_engineered_optimized.xlsx", index=False)
print("✓ File saved: df3_feature_engineered_optimized.xlsx")

# %%
# =============================================================================
# PART 4: FEATURE SELECTION & IMPORTANCE ANALYSIS
# =============================================================================
print("\n" + "="*80)
print("PART 4: FEATURE SELECTION")
print("="*80)

data = pd.read_excel('df3_feature_engineered_optimized.xlsx')

# Correlation analysis
numeric_df2 = data.select_dtypes(include=[np.number])
target_corr = numeric_df2.corr()['Curah Hujan Old'].sort_values(ascending=False)

print("\nTop 15 features by correlation:")
print(target_corr.head(15))

# %%
# =============================================================================
# PART 5: OPTIMIZED TWO-STAGE MODEL WITH HYPERPARAMETER TUNING
# =============================================================================
print("\n" + "="*80)
print("PART 5: OPTIMIZED TWO-STAGE MODELING")
print("="*80)

df4 = pd.read_excel("df3_feature_engineered_optimized.xlsx")

# Create binary target (rain event)
df4["Rain_Event"] = (df4["Curah Hujan Old"] > 0).astype(int)

print(f"Rain events: {df4['Rain_Event'].sum()} ({df4['Rain_Event'].mean()*100:.1f}%)")
print(f"No rain events: {(1-df4['Rain_Event']).sum()} ({(1-df4['Rain_Event']).mean()*100:.1f}%)")

# Feature selection - exclude original columns
feature_cols = df4.columns.drop([
    "Tanggal",
    "Curah Hujan Old",
    "Curah Hujan (mm)",
    "Rain_Event",
    "Kelembapan Rata-rata",
    "Temperatur Rata-rata",
    "Temperatur Maksimum",
    "Temperatur Minimum",
    "Lamanya Penyinaran Matahari",
    "Kecepatan Angin Rata-rata",
    "is_dry"  # Remove this as it's redundant with Rain_Event
])

X = df4[feature_cols]
y_cls = df4["Rain_Event"]

# Time series split
tscv = TimeSeriesSplit(n_splits=5)
train_idx, test_idx = list(tscv.split(X))[-1]

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train_cls, y_test_cls = y_cls.iloc[train_idx], y_cls.iloc[test_idx]

print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

# %%
# =============================================================================
# STAGE 1: RAIN EVENT CLASSIFIER WITH OPTIMIZATION
# =============================================================================
print("\n" + "-"*80)
print("STAGE 1: RAIN EVENT CLASSIFICATION")
print("-"*80)

# Quick feature importance to select top features
print("\nTraining quick RF for feature selection...")
rf_quick = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_quick.fit(X_train, y_train_cls)

# Get feature importance
feature_importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_quick.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 most important features:")
print(feature_importance_df.head(20))

# Select top 40 features
top_n_features = 40
top_features = feature_importance_df.head(top_n_features)['feature'].tolist()
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(f"\n✓ Using top {top_n_features} features for modeling")

# Train optimized classifier with class weighting
print("\nTraining optimized Random Forest classifier...")
clf = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    min_samples_leaf=3,
    min_samples_split=5,
    class_weight='balanced',  # Handle imbalance
    random_state=42,
    n_jobs=-1
)

clf.fit(X_train_selected, y_train_cls)

# Predict probabilities
y_pred_prob = clf.predict_proba(X_test_selected)[:, 1]

# Find optimal threshold
print("\nOptimizing threshold...")
precisions, recalls, thresholds = precision_recall_curve(y_test_cls, y_pred_prob)

# Calculate F1 scores for all thresholds
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1] + 1e-10)
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]

print(f"✓ Optimal threshold: {best_threshold:.3f}")
print(f"✓ F1-score at optimal threshold: {f1_scores[best_threshold_idx]:.4f}")

# Apply optimal threshold
y_pred_cls = (y_pred_prob >= best_threshold).astype(int)

# Calculate metrics
acc = accuracy_score(y_test_cls, y_pred_cls)
f1 = f1_score(y_test_cls, y_pred_cls)

print(f"\n✓ Stage 1 Results:")
print(f"  - Accuracy: {acc:.4f}")
print(f"  - F1-Score: {f1:.4f}")

# %%
# =============================================================================
# STAGE 2: RAIN AMOUNT REGRESSION WITH ENSEMBLE
# =============================================================================
print("\n" + "-"*80)
print("STAGE 2: RAIN AMOUNT REGRESSION (ENSEMBLE)")
print("-"*80)

# Filter only rainy days
df4_rain = df4[df4["Curah Hujan Old"] > 0].copy()

X_rain = df4_rain[top_features]  # Use same selected features
y_rain = np.log1p(df4_rain["Curah Hujan Old"])  # Log transform for better distribution

train_idx_r, test_idx_r = list(tscv.split(X_rain))[-1]

X_train_r, X_test_r = X_rain.iloc[train_idx_r], X_rain.iloc[test_idx_r]
y_train_r, y_test_r = y_rain.iloc[train_idx_r], y_rain.iloc[test_idx_r]

print(f"\nRainy days - Train: {len(X_train_r)}, Test: {len(X_test_r)}")

# Train multiple models for ensemble
print("\nTraining ensemble of regressors...")

# Model 1: XGBoost (optimized)
print("  - Training XGBoost...")
xgb_reg = XGBRegressor(
    n_estimators=1000,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    reg_alpha=0.5,
    reg_lambda=1.5,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)
xgb_reg.fit(X_train_r, y_train_r)

# Model 2: LightGBM
print("  - Training LightGBM...")
lgb_reg = LGBMRegressor(
    n_estimators=1000,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.5,
    reg_lambda=1.5,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_reg.fit(X_train_r, y_train_r)

# Model 3: Gradient Boosting
print("  - Training Gradient Boosting...")
gb_reg = GradientBoostingRegressor(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    random_state=42
)
gb_reg.fit(X_train_r, y_train_r)

print("✓ All regressors trained!")

# %%
# =============================================================================
# FINAL PREDICTION WITH ENSEMBLE
# =============================================================================
print("\n" + "-"*80)
print("GENERATING FINAL PREDICTIONS")
print("-"*80)

final_pred = []

for i in range(len(X_test_selected)):
    prob = y_pred_prob[i]
    
    if prob < best_threshold:
        # Predicted no rain
        final_pred.append(0)
    else:
        # Predicted rain - use ensemble of regressors
        xgb_pred = xgb_reg.predict(X_test_selected.iloc[[i]])[0]
        lgb_pred = lgb_reg.predict(X_test_selected.iloc[[i]])[0]
        gb_pred = gb_reg.predict(X_test_selected.iloc[[i]])[0]
        
        # Weighted ensemble (XGB and LGB usually perform better)
        ensemble_log = (0.4 * xgb_pred + 0.4 * lgb_pred + 0.2 * gb_pred)
        rain_mm = np.expm1(ensemble_log)
        
        # Smooth probability weighting
        prob_weight = np.sqrt(prob)  # Square root for smoother transition
        final_pred.append(prob_weight * rain_mm)

final_pred = np.array(final_pred)
y_true = df4.iloc[test_idx]["Curah Hujan Old"]

# %%
# =============================================================================
# EVALUATION & RESULTS
# =============================================================================
print("\n" + "="*80)
print("FINAL RESULTS - OPTIMIZED TWO-STAGE MODEL")
print("="*80)

mae = mean_absolute_error(y_true, final_pred)
mse = mean_squared_error(y_true, final_pred)
rmse = np.sqrt(mse)

# Calculate improvement metrics
mape = np.mean(np.abs((y_true - final_pred) / (y_true + 1))) * 100

print(f"\n✓ FINAL METRICS:")
print(f"  - MAE:  {mae:.4f} mm")
print(f"  - MSE:  {mse:.4f}")
print(f"  - RMSE: {rmse:.4f} mm")
print(f"  - MAPE: {mape:.2f}%")

# Additional statistics
print(f"\n✓ PREDICTION STATISTICS:")
print(f"  - Actual mean rain:    {y_true.mean():.2f} mm")
print(f"  - Predicted mean rain: {final_pred.mean():.2f} mm")
print(f"  - Actual max rain:     {y_true.max():.2f} mm")
print(f"  - Predicted max rain:  {final_pred.max():.2f} mm")

# Save results
result = df4.iloc[test_idx][["Tanggal"]].copy()
result["Actual_Rain_mm"] = y_true.values
result["Predicted_Rain_mm"] = final_pred
result["Error"] = abs(y_true.values - final_pred)
result["Error_Percentage"] = (result["Error"] / (result["Actual_Rain_mm"] + 1)) * 100

result.to_excel("optimized_two_stage_prediction_result.xlsx", index=False)
print("\n✓ Results saved to: optimized_two_stage_prediction_result.xlsx")

# Show worst predictions
print("\n✓ Top 10 WORST PREDICTIONS:")
worst = result.nlargest(10, 'Error')[['Tanggal', 'Actual_Rain_mm', 'Predicted_Rain_mm', 'Error']]
print(worst.to_string(index=False))

# Show best predictions
print("\n✓ Top 10 BEST PREDICTIONS:")
best = result.nsmallest(10, 'Error')[['Tanggal', 'Actual_Rain_mm', 'Predicted_Rain_mm', 'Error']]
print(best.to_string(index=False))

print("\n" + "="*80)
print("OPTIMIZATION COMPLETE!")
print("="*80)
print("\nKey Improvements Implemented:")
print("1. ✓ Advanced feature engineering (seasonal, interaction features)")
print("2. ✓ Feature selection (top 40 most important features)")
print("3. ✓ Optimized threshold selection using F1-score")
print("4. ✓ Class balancing with 'balanced' weights")
print("5. ✓ Ensemble of 3 regressors (XGBoost, LightGBM, GradientBoosting)")
print("6. ✓ Smooth probability weighting")
print("7. ✓ Enhanced hyperparameters")
print("\nCompare these results with your baseline:")
print("  Baseline MAE:  6.3530 mm")
print("  Baseline RMSE: 15.1230 mm")
print("  Baseline Acc:  0.5943")

PART 1: DATA LOADING & CLEANING
Initial data shape: (4020, 11)
Strip values found: 497
8888 values found: 209

PART 2: MISSING VALUE IMPUTATION WITH RANDOM FOREST

Top 10 correlations with target:
Curah Hujan (mm)                      1.000000
Kelembapan Rata-rata                  0.399640
Arah Angin Saat Kecepatan Maksimum    0.085265
Arah Angin Terbanyak (°)              0.002441
Kecepatan Angin Maksimum             -0.009466
Kecepatan Angin Rata-rata            -0.032244
Lamanya Penyinaran Matahari          -0.199244
Temperatur Maksimum                  -0.319690
Temperatur Rata-rata                 -0.388173
Temperatur Minimum                   -0.402973
Name: Curah Hujan (mm), dtype: float64

Imputing 706 missing target values...
✓ File saved: Ready_To_Use_Updated_RF.xlsx
✓ Missing values after imputation: 0

PART 3: ADVANCED FEATURE ENGINEERING
Creating lag features...
Creating rolling statistics...
Creating seasonal features...
Creating interaction features...
Creating dry spell